[View in Colaboratory](https://colab.research.google.com/github/hdh7485/cnn-text-classification-tf/blob/master/NavScript.ipynb)

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [18]:
!ps ax|grep python

   89 ?        Sl     0:01 /usr/bin/python2 /usr/local/bin/jupyter-notebook -y --no-browser --log-level=DEBUG --debug --NotebookApp.allow_origin="*" --NotebookApp.log_format="%(message)s" --NotebookApp.token= --Session.key="" --Session.keyfile="" --ContentsManager.untitled_directory="Untitled Folder" --ContentsManager.untitled_file="Untitled File" --ContentsManager.untitled_notebook="Untitled Notebook" --NotebookNotary.algorithm="sha1" --KernelManager.autorestart=True --MultiKernelManager.default_kernel_name="python2" --ip="127.0.0.1" --port=9000 --port-retries=0 --notebook-dir="/content" --NotebookNotary.algorithm=sha256 --NotebookNotary.secret_file=/content/datalab/.config/notary_secret --NotebookApp.base_url=/tun/m/gpu-cc67aecd-ee17-4bb8-8158-8ded892e6622/
   97 ?        Ssl    1:16 /usr/bin/python3 -m ipykernel_launcher -f /content/.local/share/jupyter/runtime/kernel-b880ed7c-d88e-415c-a2f8-571629d14dce.json
  875 pts/0    Ss+    0:01 /bin/sh -c ps ax|grep python
  877 pts/0    S+ 

In [19]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [20]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.1 GB  I Proc size: 2.0 GB
GPU RAM Free: 548MB | Used: 10891MB | Util  95% | Total 11439MB


# Universal Sentence Encoder


<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>


This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [21]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
def rmse(predictions, targets):
  return np.sqrt(((predictions - targets)**2).mean())

In [0]:
# 미리 학습된 모듈의 url
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/1" #@param ["https://tfhub.dev/google/universal-sentence-encoder/1", "https://tfhub.dev/google/universal-sentence-encoder-large/1"]

In [0]:
scripts = ["[SEARCH FROM:WEATHERFORECAST WHERE:HERE WHEN:AFTERNOON]",
         "[SEARCH FROM:TRAFFIC WHERE:ONROUTE]",
         "[SEARCH FROM:TRAFFICCAMERA WHERE:[SEARCH GEOCODE WHERE:”US-101 and Bayshore Blvd”]]",
         "[SEARCH FROM:GASSTATION WHERE:NEARBY WITH:RESTROOM]",
         "[SEARCH ONE FROM:COFFEESHOP WHERE:ALONGROUTE]",
         "[SEARCH ONE FROM:OFFROADPARKING WHERE:DESTINATION RANGE:500M WITH:[SORT PRICE ASC]]",
         "[SEARCH ONE FROM:SUPERMARKET WHERE:ONROUTE WITH:CHARGINGSTATION]",
         "[SEARCH ONE FROM:PARKING WITH:CREDITCARD WITH:VALETSERVICE]",
         "[ROUTE TO:[SEARCH KEYWORD:”San Francisco Museum of Modern Art”]]",
         "[ROUTE INFO:ETA]",
         "[ROUTE ALTROUTE]",
         "[ROUTE ALTROUTE USE:[SEARCH LINKS:ROUTE]]",
         "[MODE ROUTEOVERVIEW]",
         "[MODE GUIDANCE WITH:[ROUTE TO:[SEARCH KEYWORD:”Downtown Berkeley”]]]",
         "[MODE DRIVERANGE]",
         "[MODE DRIVERANGE TO:[SEARCH KEYWORD:”10AM MEETING” FROM:SCHEDULE WHEN:10AM] WITH:[VOICERESPONSE TEMPLATE:YES/NO*]]",
         "[MODE TRAFFIC [SEARCH FROM:TRAFFIC WHERE:[SEARCH KEYWORD:”Bay Bridge”]] WITH:[VOICERESPONSE TEMPLATE:””*]",
         "[MODE SPEEDCAMERA WHERE:ONROUTE WITH:[VOICERESPONSE TEMPLATE:””*]]",
         "[MODE WEATHERFORECAST WHERE:[SEARCH KEYWORD:”Oakland”] WHEN:TOMORROW]"
          ]

# Compute a representation for each message, showing various lengths supported.
messages = ["What's the something for this time?",
            "What's the something like on place?",
            "Show me a something on geocode and geocode.",
            "Can you find me a something with something where?",
            "Find a something where.",
            "Find the cheapest indoor parking within 500 meters of my destination.",
            "Okay, can you find me a supermarket on my route that has a charging station?",
            "Find parking near destination that accepts credit cards and has a valet service.",
            "Navigate to San Francisco Museum of Modern Art.",
            "What's my ETA to destination?",
            "Show me alternative routes.",
            "Reroute using I-580 East.",
            "Show route overview.",
            "Drive to Downtown Berkeley.",
            "What's my drive range?",
            "Can I make tomorrow's 10am meeting without recharging?",
            "What's traffic like on the Bay Bridge?",
            "Are there any speed cameras on my route?",
            "Will it rain tomorrow in Oakland?"
            ]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  #임베딩 시작
  message_embeddings = session.run(embed(messages))

In [26]:
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: What's the something for this time?
Embedding size: 512
Embedding: [0.10562784969806671, 0.007974918931722641, -0.05560705065727234, ...]

Message: What's the something like on place?
Embedding size: 512
Embedding: [0.09381017833948135, 0.007413230370730162, -0.015313537791371346, ...]

Message: Show me a something on geocode and geocode.
Embedding size: 512
Embedding: [0.09865029156208038, -0.03687693923711777, -0.017361536622047424, ...]

Message: Can you find me a something with something where?
Embedding size: 512
Embedding: [0.09483873844146729, -0.019978588446974754, -0.05640304088592529, ...]

Message: Find a something where.
Embedding size: 512
Embedding: [0.09537440538406372, -0.026791611686348915, -0.07760915905237198, ...]

Message: Find the cheapest indoor parking within 500 meters of my destination.
Embedding size: 512
Embedding: [0.03883657604455948, 0.06337009370326996, -0.005243263673037291, ...]

Message: Okay, can you find me a supermarket on my route that ha

In [0]:
# Compute a representation for each message, showing various lengths supported.
test_message = ["Where is a gas station with a restroom nearby?",
               "Find routes to New York University."]
test_labels = [3, 8]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  #임베딩 시작
  test_message_embeddings = session.run(embed(test_message))
  #test_message_embedding = test_message_embeddings[0]

In [38]:
for test_message_embedding, test_label in zip(test_message_embeddings, test_labels):
  minimum = 100
  minimum_index = 0
  for i, message_embedding in enumerate(message_embeddings):
    error = rmse(np.array(message_embedding), np.array(test_message_embedding))
    if minimum > error:
      minimum = error
      minimum_index = i

  print("Minimum RMSE value: {}".format(minimum))
  print("Most similar script: {}".format(scripts[minimum_index]))
  print("Estimation: {}".format(minimum_index))
  print("Answer: {}\n".format(test_label))
  #프로파일 데이터의 명사들 변환 필요
  #테스트 데이터의 명사들 변환 필요
  #다른 문장들도 테스트 해볼 것.

Minimum RMSE value: 0.037473857402801514
Most similar script: [SEARCH ONE FROM:SUPERMARKET WHERE:ONROUTE WITH:CHARGINGSTATION]
Estimation: 6
Answer: 3

Minimum RMSE value: 0.037789326161146164
Most similar script: [MODE GUIDANCE WITH:[ROUTE TO:[SEARCH KEYWORD:”Downtown Berkeley”]]]
Estimation: 13
Answer: 8

